Notes: 

1. Complete full game is been retireved code is done, but the column names are not accurate need to check it.

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from time import sleep
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from lxml.html import fromstring
import requests
from itertools import cycle
import traceback
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sqlalchemy import create_engine
import datetime
import re
import os
import pickle


In [2]:
# Creating Functions

#### Getting Proxy

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr'):
        if (i.xpath('.//td[7][contains(text(),"yes")]')) and (i.xpath('.//td[5][contains(text(),"elite proxy")]')):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [3]:
### FInding good_proxy

def get_good_proxy():
    proxies = get_proxies()
    proxy_pool = cycle(proxies)
    url = 'https://httpbin.org/ip'
    url2 = 'https://www.google.com/maps/dir///@13.115399,77.6192442,15z/data=!4m5!4m4!1m1!4e2!1m0!3e0'

    print("Length of Proxies List",len(proxies))

    good_Proxy = 0

    for i in range(1,len(proxies)):
        #Get a proxy from the pool
        proxy = next(proxy_pool)
        #print("Request #%d"%i)
        try:
            response = requests.get(url,proxies={"http": proxy, "https": proxy}, timeout=10)
            #print(response.json())
            #print(response.status_code)
            response2 = requests.get(url2, proxies={"http": proxy, "https": proxy}, timeout=10)
            print(response2.status_code)

        except:
            print("Skipping. Connnection error")


        else:
            if int(response2.status_code) == 200:
                good_Proxy = proxy
                print("Good Proxy: ", good_Proxy)
                print("sucess")
                break
            else:
                print("Error")
        
            return good_Proxy
    

            

In [4]:
####### Function that returns links for all montgs where NBA took place

def month_func(driver):
    
    month_links =[]
    
    ############ default range is 10 as we have 9 months (NBA games)
    for i in range(1,10):
        

        ##### NBA Month page (links)
        try:
            link_xpath = '//*[@id="content"]/div[2]/div[{}]/a'.format(i)
            link = driver.find_element_by_xpath(link_xpath).get_attribute('href')
            month_links.append(link)
        except:
            pass
 
        
    return month_links

In [5]:
def boxscore_func(driver):
    
    boxscore_links = []
    
     ##### Box Score page (links)
    # Find all elements matching the XPath
    boxscores_xpath = '//*[@id="schedule"]/tbody//td[6]/a'
    elements = driver.find_elements_by_xpath(boxscores_xpath)

    # Extract href attributes from the elements
    boxscore_links = [element.get_attribute('href') for element in elements]
    
    return boxscore_links

In [6]:
#### New Version

def read_stats(element):
    html_content = element[0].get_attribute('outerHTML')
    df = pd.read_html(html_content)[0]
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

def get_scores(driver, BoxScore_URL):
    driver.get(BoxScore_URL)
    driver.implicitly_wait(10)
    tables = driver.find_elements(By.XPATH, '//*[@id="div_line_score"]//table')
    games = []

    for table in tables:
        headers = [header.text for header in table.find_elements(By.XPATH, './/thead/tr/th')]
        headers = [h for h in headers if h != 'Scoring']  # More safe way to remove 'Scoring'
        
        rows = table.find_elements(By.XPATH, './/tbody/tr')
        data = [[data.text for data in row.find_elements(By.XPATH, './/th | .//td')] for row in rows]
        linescore_df = pd.DataFrame(data, columns=headers)
        linescore_df.rename(columns={'T': 'total', ' ': 'team'}, inplace=True)
        teams = list(linescore_df['team'])
        
        summaries = []
        for team in teams:
            try:
                basic_element = driver.find_elements(By.XPATH, f'//*[@id="box-{team}-game-basic"]')
                advanced_element = driver.find_elements(By.XPATH, f'//*[@id="box-{team}-game-advanced"]')
                basic = read_stats(basic_element)
                advanced = read_stats(advanced_element)

                # Combine stats
                total_stats = pd.concat([basic.iloc[-1], advanced.iloc[-1]])
                total_stats.index = [idx[1] if isinstance(idx, tuple) else idx for idx in total_stats.index]
                total_stats.index = total_stats.index.str.lower() + '_total'

                max_stats = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
                max_stats.index = [idx[1] if isinstance(idx, tuple) else idx for idx in max_stats.index]
                max_stats.index = max_stats.index.str.lower() + '_max'

                summary = pd.concat([total_stats, max_stats])
                summaries.append(summary)
            except Exception as e:
                print(f"Error processing team {team}: {e}")
                continue  # Skip to next team if error
        
        if summaries:
            try:
                summary_df = pd.concat(summaries, axis=1).T
                summary_df.drop(columns=['bpm'], errors='ignore', inplace=True)
                game_df = pd.concat([summary_df, linescore_df], axis=1)
                game_df['home'] = [0, 1]

                game_opp = game_df.iloc[::-1].reset_index(drop=True)
                game_opp.columns = [f'{col}_opp' for col in game_opp.columns]

                full_game = pd.concat([game_df, game_opp], axis=1)
                full_game['date'] = pd.to_datetime(os.path.basename(BoxScore_URL)[:8], format='%Y%m%d')
                full_game['season'] = full_game['date'].dt.year
                full_game['won'] = full_game['total'] > full_game['total_opp']
                full_game = full_game.loc[:, ~full_game.columns.duplicated(keep='first')]

                games.append(full_game)
            except Exception as e:
                print(f"Error processing game data: {e}")

    return games


In [7]:
#### Opening Driver

def open_driver(year):
    PROXY = get_good_proxy()
    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": PROXY,
        "ftpProxy": PROXY,
        "sslProxy": PROXY,

        "proxyType": "MANUAL",

    }

    # object of Options class
    op = Options()
    op.headless = True ## opening chrome in background

    #### Old method
#     # set chromedriver.exe path
#     driver = webdriver.Chrome(options=op)
#     driver.maximize_window()
#     sleep(2)
    
    #### New Way
    driver = webdriver.Chrome("chromedriver-win64\chromedriver.exe", options=op)
    driver.maximize_window()
    sleep(2)
    
    
    BasketBallReferenceLink = 'https://www.basketball-reference.com/leagues/NBA_{0}_games.html'.format(year)
    driver.get(BasketBallReferenceLink)
    sleep(3)
    
    ## Calling Month_func
    month_links = month_func(driver)
    
    ## Calling Boxscore_func
    boxscore_links_list = []
    for month_link in month_links:
        driver.get(month_link)
        sleep(3)
        boxscore_links = boxscore_func(driver)
        boxscore_links_list.append(boxscore_links)
    
    
    ######## LineScores data 
    linescore_df_list = []
    ## BoxScore_URL_list_of_list = [['https://www.basketball-reference.com/boxscores/202012220BRK.html','https://www.basketball-reference.com/boxscores/202203280CHO.html']]
    BoxScore_URL_list_of_list = boxscore_links_list
    
    ### Counter
    counter = 0

    
    
    for BoxScore_URL_list in BoxScore_URL_list_of_list:
        print("Length of list: ", len(BoxScore_URL_list))
            
        for BoxScore_URL in BoxScore_URL_list:
            
            
            # Increment the counter
            counter += 1
            
            ## print(str(BoxScore_URL))

            driver.get(str(BoxScore_URL))
            sleep(3)
            linescore_df = get_scores(driver, BoxScore_URL)
            linescore_df_list.append(linescore_df)
            
            # Check if the counter is at a multiple of 50
            if counter % 50 == 0:
                print(f"{counter} links are saved")
        
    print("Hello world")
    # close browser after our manipulations
    driver.close()
    
    ## return month_links,BoxScore_URL_list_of_list,linescore_df_list
    return linescore_df_list

In [8]:
## Seasons

## season = [2018,2019,2020,2021,2022,2023]

season = [2017]


## Create a list with the length of 'season'
len_of_season = list(range(len(season)))


combined_df_list = []

from datetime import datetime


start_now = datetime.now()
start_time = start_now.strftime("%H:%M:%S")
print("start Time of Mini_data =", start_time)
    

for i,j in zip(season, len_of_season):
    
    print("Season - {0}".format(i))
    start_now = datetime.now()
    start_time = start_now.strftime("%H:%M:%S")
    print("start Time of Mini_data =", start_time)
    

    combined_dataframes = open_driver(i)
    
    # Assume complete_dfs is your list of lists of DataFrames
    flattened_list_of_dfs = [df for sublist in combined_dataframes for df in sublist]

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(flattened_list_of_dfs, ignore_index=True)

    combined_df_list.append(combined_df)
    
    #### Save the list of DataFrames to a pickle file
    with open('dataframes_list_season_-{0}.pkl'.format(i), 'wb') as file:
        pickle.dump(combined_df, file)
        
    print('dataframes_list_season-{0}.pkl saved'.format(i))

end_now = datetime.now()
end_time = end_now.strftime("%H:%M:%S")
print("End Time  = ", end_time)

start Time of Mini_data = 07:59:20
Season - 2017
start Time of Mini_data = 07:59:20
Length of Proxies List 20
Skipping. Connnection error
Skipping. Connnection error
Skipping. Connnection error
Skipping. Connnection error
Skipping. Connnection error
Skipping. Connnection error
Skipping. Connnection error
200
Good Proxy:  189.240.60.166:9090
sucess
Length of list:  45
Length of list:  229
50 links are saved
100 links are saved
150 links are saved
200 links are saved
250 links are saved
Length of list:  232
300 links are saved
350 links are saved
400 links are saved
450 links are saved
500 links are saved
Length of list:  223
550 links are saved
600 links are saved
650 links are saved
700 links are saved
Length of list:  165
750 links are saved
800 links are saved
850 links are saved
Length of list:  241
900 links are saved
950 links are saved
1000 links are saved
1050 links are saved
1100 links are saved
Length of list:  140
1150 links are saved
1200 links are saved
1250 links are saved

In [9]:
########## Need to extract line scores, Basic Box Score Stats & Advanced Box Score Stats (In Starts consider only summaries of the match) (For home & away always 1st team standing is away team.)